In [1]:
import random
import numpy as np

In [2]:
class FoodSource:
  def __init__(self, bounds, obj_function):
    self.position = np.array([random.uniform(bound[0], bound[1]) for bound in bounds])
    self.obj_value = obj_function(self.position)
    self.fitness = self.calculate_fitness()
    self.trial = 0
    self.probability = 0

  def __lt__(self, other):
    return self.obj_value < other.obj_value

  def __gt__(self, other):
    return self.fitness > other.fitness

  def calculate_fitness(self):
    if self.obj_value >= 0:
      return 1 / (1 + self.obj_value)
    return 1 + abs(self.obj_value) 

In [3]:
def rastrigin(x):
    A = 10
    n = len(x)
    return A*n + sum(x_i**2 - A*np.cos(2*np.pi*x_i) for x_i in x)
    #return x[0]**2 + x[1]**2

In [4]:
def rosenbrock(x):
    a = 1
    b = 100
    return (a - x[0]) ** 2 + b * (x[1] - x[0]**2)**2

In [5]:
bounds = np.array([(-5.12, 5.12), (-5.12, 5.12)])

In [6]:
def generate_new_solution(i, population, lb, ub, obj_function):

  j = random.randrange(len(bounds))
  #print("j = ", j)
  partner = random.randrange(len(population))
  #print("partner = ", partner)

  while i == partner:
    partner = random.randrange(len(population))

  phi = random.uniform(-1, 1)

  old_fitness = population[i].fitness
  old_objective_value = population[i].obj_value

  old_j = population[i].position[j]
  new_j = population[i].position[j] + phi * (population[i].position[j] - population[partner].position[j])

  #print("New_j ---> ", new_j)
  population[i].position[j] = new_j
  population[i].position[j] = np.clip(population[i].position[j], lb[j], ub[j])

  print(old_j, population[i].position[j])  

  new_objective_value = obj_function(population[i].position)
   
  population[i].obj_value = new_objective_value
  population[i].fitness = population[i].calculate_fitness()

  #print(f"old_fitness = ${old_fitness}, new_fitness = ${new_fitness}")
  #print(f"old_j = ${old_j}, new_j = ${new_j}")

  if population[i].fitness > old_fitness:
    print("Poboljsano resenje")
    population[i].trial = 0
  
  else:
    population[i].position[j] = old_j
    population[i].obj_value = old_objective_value
    population[i].fitness = old_fitness
    population[i].trial = population[i].trial + 1 

  

In [7]:
POPULATION_SIZE = 50
NUM_OF_ITERATIONS = 100
LIMIT = 5

lower_bound = np.array([x[0] for x in bounds])
upper_bound = np.array([x[1] for x in bounds])


population = [FoodSource(bounds, rastrigin) for _ in range(POPULATION_SIZE)]

best_solution = min(population)

for k in range(NUM_OF_ITERATIONS):
  #### Employed Bee Phase ####
  for i in range(POPULATION_SIZE):
    generate_new_solution(i, population, lower_bound, upper_bound, rastrigin)

  #### Onlooker Bee Phase ####
  max_fitness = max([population[i].fitness for i in range(POPULATION_SIZE)])
  for i in range(POPULATION_SIZE):
    population[i].probability = 0.9 * (population[i].fitness / max_fitness) + 0.1


  m = 0
  n = 0

  while m < POPULATION_SIZE and n < POPULATION_SIZE:
    rand_value = random.uniform(0, 1)
    if rand_value < population[n].probability:
      generate_new_solution(n, population, lower_bound, upper_bound, rastrigin)
      m += 1
    n = (n % POPULATION_SIZE) + 1

  best_solution = min(min(population), best_solution)

  #### Scout Bee Phase ####
  max_index = -1
  max_trial = float('-inf')
  for index, p in enumerate(population):
    if p.trial > max_trial:
      max_trial = p.trial
      max_index = index


  if population[max_index].trial > LIMIT:
    population[max_index] = FoodSource(bounds, rastrigin)

  best_solution = min(min(population), best_solution)


print("position: ", best_solution.position[0], best_solution.position[1])
print("objective value: ", best_solution.obj_value)
print("fitness value: ", best_solution.fitness)

Streaming output truncated to the last 5000 lines.
-0.002761596753879436 -0.0030610082193692576
-0.031026473493657725 2.4322952885911175
-0.10208852996658996 -0.07382579336976067
Poboljsano resenje
-0.04371660719055154 -0.33535707017640054
1.0021879117574197 0.0017646454456825555
Poboljsano resenje
-0.047065574626336315 -0.017818904513496676
Poboljsano resenje
2.1418659998482195 2.3212470162655268
-0.0022113336771520595 -0.0024269856057788004
-2.1261505900676356 -2.130525776396277
-2.0087939652893434 -0.9352824313471617
Poboljsano resenje
-0.0669419768467609 -0.6536407934563586
0.0021342702698845866 -0.12495225562162521
-0.002668828379782829 0.2616173651681992
-0.002058353611996383 0.8414699466233583
-0.004583826326506912 -0.9910188207380531
-1.0580667716803258 -1.0723510815605373
-0.022819440140075503 -0.06244510294995714
-0.011315979476089725 -0.1573814765027356
-2.084912181166132e-05 0.5387864737542524
-0.994425598449486 -0.7206009581587389
0.05063832693432313 0.01658461019533304
Po